In [53]:
%matplotlib inline
import random
import numpy as np
import networkx as nx
from cascade import generate_cascade_old
from graph_generator import grid_2d, add_p_and_delta

In [30]:
g = grid_2d(10)
g = add_p_and_delta(g, p=0.7, d=1)

In [31]:
source = random.choice(g.nodes())

rands = np.random.rand(g.number_of_edges())
active_edges = [(u, v) for (u, v), r in zip(g.edges_iter(), rands) if g[u][v]['p'] >= r]
induced_g = nx.Graph()
induced_g.add_edges_from(active_edges)
for u, v in induced_g.edges_iter():
    induced_g[u][v]['d'] = g[u][v]['d']

In [32]:
induced_g.number_of_edges() / g.number_of_edges()

0.6833333333333333

In [33]:
es1 = set(map(lambda e: tuple(sorted(e)), g.edges()))
es2 = set(map(lambda e: tuple(sorted(e)), induced_g.edges()))

In [34]:
assert es2.issubset(es1)

In [35]:
infection_times = nx.shortest_path_length(induced_g, source=source, weight='d')

In [36]:
assert infection_times[source] == 0

In [40]:
for n in g.nodes_iter():
    if n not in infection_times:
        infection_times[n] = float('inf')

In [41]:
assert len(infection_times) == g.number_of_nodes()

In [46]:
def generate_cascade(g):
    source = random.choice(g.nodes())

    rands = np.random.rand(g.number_of_edges())
    active_edges = [(u, v) for (u, v), r in zip(g.edges_iter(), rands) if g[u][v]['p'] >= r]
    induced_g = nx.Graph()
    induced_g.add_edges_from(active_edges)
    for u, v in induced_g.edges_iter():
        induced_g[u][v]['d'] = g[u][v]['d']
        
    if not induced_g.has_node(source):
        infection_times = {n: float('inf') for n in g.nodes_iter()}
        infection_times[source] = 0
    else:                
        infection_times = nx.shortest_path_length(induced_g, source=source, weight='d')
        for n in g.nodes_iter():
            if n not in infection_times:
                infection_times[n] = float('inf')    
    assert infection_times[source] == 0
    assert len(infection_times) == g.number_of_nodes()
    return infection_times

In [54]:
from utils import infeciton_time2weight
from simulations import make_input
from plot_utils import plot_snapshot
pos = {n: np.array(n) for n in g.nodes()}

source, obs_nodes, infection_times, tree = make_input(g, infp, 0.05, sampling_method='late_nodes')

node2weight = infeciton_time2weight(infection_times)

plot_snapshot(g, pos, node2weight, 
              query_node=(3, 3),
              source_node=source,
              queried_nodes=obs_nodes, 
              max_node_size=500)

NameError: name 'infp' is not defined

In [49]:
%%timeit
generate_cascade(g)

1000 loops, best of 3: 1.05 ms per loop


In [51]:
%%timeit
generate_cascade_old(g)

100 loops, best of 3: 3.97 ms per loop
